In [1]:
import os
import numpy as np
from PIL import Image
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from src.Models.D_UNet import UNet2D, ResidualUNet2D
from src.configuration.config import datadict
from src.Dataset.dataset import CustomDataset2D
from src.utils.utils import custom_collate_2D

In [2]:
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
train_transform = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

In [3]:
import os
Dir = r"C:\Users\Rishabh\Documents\pytorch-3dunet\TrainingData"
image_dir = os.path.join(Dir, 'Images')
mask_dir = os.path.join(Dir, 'Masks')
data = CustomDataset2D(image_dir, mask_dir,transform = train_transform)

In [4]:
train_loader = DataLoader(
        data,
        batch_size=4,
        num_workers=0,
        pin_memory=True,
        shuffle=True,
        collate_fn=custom_collate_2D
    )

In [ ]:
ImageVolume ,Maskvolume = data[20]

In [ ]:
ImageVolume.shape

In [ ]:
Maskvolume.shape

In [ ]:
len(data)

In [ ]:
np.unique(Maskvolume)

In [ ]:
def save_prediction(image, mask):
    gray_image = image
    binary_mask = mask
    
    gray_norm = gray_image / 255.0

    # Create an RGB image with grayscale as background
    overlay = np.stack([gray_norm, gray_norm, gray_norm], axis=-1)
    
    # Define lighter colors for each class (0-8)
    colors = {
        1: [1.0, 0.6, 0.6],   # Light Red
        2: [0.6, 1.0, 0.6],   # Light Green
        3: [0.6, 0.6, 1.0],   # Light Blue
        4: [1.0, 1.0, 0.6],   # Light Yellow
        5: [1.0, 0.6, 1.0],   # Light Magenta
        6: [0.6, 1.0, 1.0],   # Light Cyan
        7: [0.8, 0.7, 1.0],   # Light Purple
        8: [1.0, 0.8, 0.6]    # Light Orange
    }
    
    # Create an RGB mask initialized with zeros
    mask_rgb = np.zeros_like(overlay)

    
    # Assign colors based on binary_mask values
    for value, color in colors.items():
        mask_rgb[binary_mask == value] = color
    
    # Define transparency level
    alpha = 0.4  # Transparency level (0-1)
    
    # Blend grayscale image with the colored mask
    blended = overlay * (1 - alpha) + mask_rgb * alpha
    blended = (blended*255).astype(np.uint8)
    image = Image.fromarray(blended)
    index = len(os.listdir('Predictions'))+1
    image.save(f'Predictions/output_{index}.jpg', quality=100)

In [ ]:
for batch_idx, (inputs, targets) in enumerate(train_loader):
    print()
    # print(inputs.shape)
    inputs = np.array(inputs)
    targets = np.array(targets)
    
    for batch in range(inputs.shape[0]):
        gray_image = inputs[batch, 1, :, :]*255
        binary_mask = targets[batch, :, :, :]
        binary_mask = np.argmax(binary_mask, axis=0)
        # print('gray_image:-',np.unique(gray_image))
        print('binary_mask:-',np.unique(binary_mask))
        if len(np.unique(binary_mask))>1:
            # print('binary_mask:-',np.unique(binary_mask))
            save_prediction(gray_image, binary_mask)

In [14]:
def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (
                (preds + y).sum() + 1e-8
            )
            # print(dice_score)

    print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
    )
    print(f"Dice score: {dice_score/len(loader)}")
    model.train()

In [5]:
from src.Models.D_UNet import UNet2D, ResidualUNet2D

In [6]:
import torch.optim as optim
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet2D(in_channels=3, out_channels=9, f_maps=128).to(device)

LEARNING_RATE = 0.001
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
# scaler = torch.cuda.amp.GradScaler()
scaler = torch.amp.GradScaler("cuda")


# lossfn = DiceLoss()
lossfn = nn.BCEWithLogitsLoss()

In [9]:
model.train()
for i in range(10000):
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
    
        optimizer.zero_grad()
        
        # Forward pass with mixed precisio/n
        # with torch.cuda.amp.autocast():
        # with torch.amp.autocast("cuda"):
        outputs = model(inputs)
        print(torch.unique(outputs))
        print(torch.unique(targets))
        loss = lossfn(outputs, targets)
        print(loss.item())
    
        # loss.backward()
        # optimizer.step()
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

tensor([0.0733, 0.0735, 0.0737,  ..., 0.7332, 0.7334, 0.7335], device='cuda:0',
       grad_fn=<Unique2Backward0>)
tensor([0., 1.], device='cuda:0', dtype=torch.float16)
0.79052734375
tensor([0.0336, 0.0339, 0.0340,  ..., 0.8592, 0.8593, 0.8595], device='cuda:0',
       grad_fn=<Unique2Backward0>)
tensor([0., 1.], device='cuda:0', dtype=torch.float16)
0.7275390625
tensor([0.0105, 0.0106, 0.0106,  ..., 0.9577, 0.9580, 0.9581], device='cuda:0',
       grad_fn=<Unique2Backward0>)
tensor([0., 1.], device='cuda:0', dtype=torch.float16)
0.69970703125


KeyboardInterrupt: 

In [15]:
check_accuracy(train_loader, model, device="cuda")

tensor(0.1818, device='cuda:0')


KeyboardInterrupt: 